In [1]:
import os
import io
import csv
import pandas as pd
import numpy as np

import psycopg2
conn = psycopg2.connect(user='airflow', password='Xypherium-0',
                        dbname='jpstat',
                        host='35.224.240.50')

In [77]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [3]:
path = os.path.join('extracts', 'monthly_reports')

In [4]:
def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = dict.fromkeys(files)
        parent = reduce(dict.get, folders[:-1], dir)
        parent[folders[-1]] = subdir
    return dir

In [5]:
filepathlist = []
for dirpath, subdir, files in os.walk(path):
    for filename in files:
        if filename.endswith('.xls'):
            filepathlist.append(os.path.join(dirpath, filename))

In [6]:
%%time
tableshape_list = []
df_dict = {}
for filepath in filepathlist:
    sheets = pd.ExcelFile(filepath).sheet_names
    
    df = pd.read_excel(filepath, sheet_name=len(sheets)-1)
    filepath_list = filepath.split('\\')
    description = [filepath_list[2] + '-' + filepath_list[3], filepath_list[4], df.shape]
    
    if len(sheets) > 1:
        print(filepath_list[2] + '-' + filepath_list[3] + ' ' + filepath_list[4] + ' has ' + str(len(sheets)) + ' sheets.')
    
    tableshape_list.append(description)

2013-07Jul 1.xls has 2 sheets.
2013-07Jul 3-3.xls has 2 sheets.
2013-08Aug 1.xls has 2 sheets.
2013-08Aug 3-3.xls has 2 sheets.
2013-09Sep 1.xls has 2 sheets.
2013-09Sep 3-3.xls has 2 sheets.
2013-10Oct 1.xls has 2 sheets.
2013-10Oct 3-3.xls has 2 sheets.
2013-11Nov 1.xls has 2 sheets.
2013-11Nov 3-3.xls has 2 sheets.
2013-12Dec 1.xls has 2 sheets.
2013-12Dec 3-3.xls has 2 sheets.
2014-01Jan 1.xls has 2 sheets.
2014-01Jan 3-3.xls has 2 sheets.
2014-02Feb 1.xls has 2 sheets.
2014-02Feb 3-3.xls has 2 sheets.
2014-03Mar 1.xls has 2 sheets.
2014-03Mar 3-3.xls has 2 sheets.
2014-04Apr 1.xls has 2 sheets.
2014-04Apr 3-3.xls has 2 sheets.
2014-05May 1.xls has 2 sheets.
2014-05May 3-3.xls has 2 sheets.
2014-06Jun 1.xls has 2 sheets.
2014-06Jun 3-3.xls has 2 sheets.
2014-07Jul 1.xls has 2 sheets.
2014-07Jul 3-3.xls has 2 sheets.
2014-08Aug 1.xls has 2 sheets.
2014-08Aug 3-3.xls has 2 sheets.
2014-09Sep 1.xls has 2 sheets.
2014-09Sep 3-3.xls has 2 sheets.
2014-10Oct 1.xls has 2 sheets.
2014-10Oc

In [7]:
tableshape_df = pd.DataFrame(tableshape_list, columns=['year-month', 'file', 'shape'])

In [8]:
tableshape_df.info

<bound method DataFrame.info of      year-month     file       shape
0    2006-01Jan  1.xls    (88, 13)  
1    2006-01Jan  2.xls    (1285, 12)
2    2006-02Feb  1.xls    (88, 13)  
3    2006-02Feb  2.xls    (1285, 12)
4    2006-03Mar  1.xls    (88, 13)  
5    2006-03Mar  2.xls    (1285, 12)
6    2006-04Apr  1.xls    (89, 13)  
7    2006-04Apr  2.xls    (1365, 12)
8    2006-05May  1.xls    (89, 13)  
9    2006-05May  2.xls    (1365, 12)
10   2006-06Jun  1.xls    (89, 13)  
11   2006-06Jun  2.xls    (1365, 12)
12   2006-07Jul  1.xls    (89, 13)  
13   2006-07Jul  2.xls    (1365, 12)
14   2006-08Aug  1.xls    (89, 13)  
15   2006-08Aug  2.xls    (1365, 12)
16   2006-09Sep  1.xls    (89, 13)  
17   2006-09Sep  2.xls    (1365, 12)
18   2006-10Oct  1.xls    (86, 18)  
19   2006-10Oct  2.xls    (1362, 18)
20   2006-11Nov  1.xls    (86, 18)  
21   2006-11Nov  2.xls    (1362, 18)
22   2006-12Dec  1.xls    (86, 18)  
23   2006-12Dec  2.xls    (1362, 18)
24   2007-01Jan  1.xls    (86, 18)  
25   2

In [9]:
tableshape_agg_df = tableshape_df.groupby(['file', 'shape'])['year-month'].apply(list).reset_index()

In [10]:
tableshape_agg_df['count'] = [len(x) for x in tableshape_agg_df['year-month']]
[x.sort for x in tableshape_agg_df['year-month']]
tableshape_agg_df.sort_values(by='year-month')
tableshape_agg_df

file       shape  \
0   1.xls    (86, 18)     
1   1.xls    (88, 13)     
2   1.xls    (88, 18)     
3   1.xls    (89, 13)     
4   1.xls    (91, 13)     
5   1.xls    (92, 13)     
6   1.xls    (93, 13)     
7   1.xls    (105, 13)    
8   2.xls    (1285, 12)   
9   2.xls    (1362, 18)   
10  2.xls    (1365, 12)   
11  2.xls    (1396, 18)   
12  2.xls    (1399, 12)   
13  2.xls    (1416, 12)   
14  2.xls    (1557, 12)   
15  2.xls    (1575, 12)   
16  2.xls    (1681, 12)   
17  3-1.xls  (69, 70)     
18  3-2.xls  (69, 70)     
19  3-3.xls  (69, 70)     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               year-month  \
0   [2006-10Oct, 2006-11Nov, 2006-12Dec, 2007-01Jan, 2007-02Feb, 2007-03Mar]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
1   [2006-01Jan, 2006-02Feb, 2006-03Mar]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [68]:
table_1_list= [x for x in filepathlist if x.split('\\')[-1] == '1.xls']
table3_1_list = [x for x in filepathlist if '3-1' in x]
table3_2_list = [x for x in filepathlist if '3-2' in x]

In [149]:
filepath = table_1_list[0]
print(filepath)

filepath_parts = filepath.split('\\')
year = filepath_parts[2]
month = filepath_parts[3][:2]

sheets = pd.ExcelFile(filepath).sheet_names
df = pd.read_excel(filepath, sheet_name=len(sheets)-1)

messy_df = df.copy()
messy_df.reset_index(inplace=True)
messy_df = messy_df[['level_8','level_9','level_10',1,2,3]]
messy_df.columns = ['Prefecture Num', 'Prefecture', 'Prefecture_English', 'Both', 'Male', 'Female']
messy_df.dropna(axis=0, subset=['Prefecture Num', 'Both', 'Male', 'Female'], how='any', inplace=True)
messy_df.loc[messy_df['Prefecture Num'] == '全国']['Prefecture'].iloc[0]

extracts\monthly_reports\2006\01Jan\1.xls


nan

In [109]:
print(np.nan)

nan


In [12]:
def combine_clean_age_tables(table_excel_list):
    #Always take the last sheet: assume that in the absense of separate sheets, the numbers are for Japanese citizens only
    table_df_list = []

    for filepath in table_excel_list:
        filepath_parts = filepath.split('\\')
        year = filepath_parts[2]
        month = filepath_parts[3][:2]

        sheets = pd.ExcelFile(filepath).sheet_names
        df = pd.read_excel(filepath, sheet_name=len(sheets)-1)

        messy_df = df.copy()
        messy_df.reset_index(inplace=True)
        messy_df.columns = [x.replace('～', '-') for x in messy_df.iloc[13].replace({
                                                    'Prefectures': 'Prefecture Num',
                                                    '総 数 1)': 'Total',
                                                    '0～4歳': '0～4',
                                                    '90歳以上': '90 & above'
                                                    }).fillna('Prefecture')]
        messy_df.iloc[18,10] = messy_df.iloc[18,8]
        messy_df = messy_df.iloc[18:66,8:]
        messy_df.reset_index(drop=True, inplace=True)
        messy_df.dropna(axis=1, inplace=True)
        messy_df.loc[0,'Prefecture Num'] = '00'

        both_df = messy_df.iloc[:,:22]
        male_df = messy_df.iloc[:,22:44]
        female_df = messy_df.iloc[:,44:66]

        both_df['Gender'] = 'Both'
        male_df['Gender'] = 'Male'
        female_df['Gender'] = 'Female'

        this_month_df = pd.concat([both_df, male_df, female_df])
        this_month_df['Year'] = str(year)
        this_month_df['Month'] = str(month)

        if False not in this_month_df.iloc[:,2:22].applymap(np.isreal).values.flatten():
            table_df_list.append(this_month_df)
        else:
            print('Some non-numeric value found in counts.')
    
    combined_df = pd.concat(table_df_list).reset_index(drop=True)
    return combined_df

In [24]:
%%time
in_migrants_age_df = combine_clean_age_tables(table3_1_list)
out_migrants_age_df = combine_clean_age_tables(table3_2_list)

C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Wall time: 22.1 s


In [96]:
out_migrants_age_df.head(10)

Prefecture Num Prefecture   Total   0-4   5-9 10-14 15-19  20-24  25-29  \
0  00             全国         126018  7924  3182  1637  3610  20043  26958   
1  01             北海道        2561    136   77    45    103   473    441     
2  02             青森県        1119    66    38    30    34    277    186     
3  03             岩手県        992     74    31    10    37    201    167     
4  04             宮城県        2085    160   59    28    66    372    422     
5  05             秋田県        753     50    24    11    25    176    121     
6  06             山形県        736     43    27    11    35    165    139     
7  07             福島県        1386    87    25    17    48    336    253     
8  08             茨城県        2564    177   78    39    74    409    518     
9  09             栃木県        1686    102   45    32    49    304    355     

   30-34  35-39 40-44 45-49 50-54 55-59 60-64 65-69 70-74 75-79 80-84 85-89  \
0  20002  13587  7826  4881  3811  3580  3247  1722  1098  946   864   711    
1  365    296    186   112   99    98    47    27    18    11    13    9      
2  152    103    51    47    39    37    22    9     9     4     5     8      
3  155    83     50    59    39    31    23    11    5     6     5     2      
4  298    242    123   88    74    54    38    17    11    13    9     8      
5  98     71     39    27    23    25    17    14    4     10    11    3      
6  99     70     32    22    27    17    21    9     7     3     1     6      
7  186    128    87    57    44    32    23    12    12    7     16    11     
8  419    289    141   90    73    76    62    29    31    17    13    16     
9  226    166    120   60    56    51    49    26    14    12    11    5      

  90 & above Gender  Year Month  
0  387        Both   2010  01    
1  5          Both   2010  01    
2  2          Both   2010  01    
3  3          Both   2010  01    
4  3          Both   2010  01    
5  4          Both   2010  01    
6  2          Both   2010  01    
7  5          Both   2010  01    
8  13         Both   2010  01    
9  3          Both   2010  01

In [42]:
keys = ['Total', '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90 & above']
in_melted_df = pd.melt(in_migrants_age_df, id_vars=['Prefecture Num', 'Prefecture', 'Gender', 'Year', 'Month'], value_name='Inmigrants', var_name='Age bucket')
out_melted_df = pd.melt(out_migrants_age_df, id_vars=['Prefecture Num', 'Prefecture', 'Gender', 'Year', 'Month'], value_name='Outmigrants', var_name='Age bucket')

In [44]:
combined_melted_df = pd.merge(in_melted_df, out_melted_df, how='left',
                              left_on=list(in_melted_df.columns).remove('Inmigrants'), 
                              right_on=list(out_melted_df.columns).remove('Outmigrants'))

In [48]:
out_melted_df.shape[0] == combined_melted_df.shape[0]

True

In [52]:
combined_melted_df.head(20)

Prefecture Num Prefecture Gender  Year Month Age bucket Inmigrants  \
0   00             全国         Both   2010  01    Total      126018      
1   01             北海道        Both   2010  01    Total      2288        
2   02             青森県        Both   2010  01    Total      954         
3   03             岩手県        Both   2010  01    Total      866         
4   04             宮城県        Both   2010  01    Total      2229        
5   05             秋田県        Both   2010  01    Total      666         
6   06             山形県        Both   2010  01    Total      646         
7   07             福島県        Both   2010  01    Total      1318        
8   08             茨城県        Both   2010  01    Total      2582        
9   09             栃木県        Both   2010  01    Total      1949        
10  10             群馬県        Both   2010  01    Total      1513        
11  11             埼玉県        Both   2010  01    Total      9053        
12  12             千葉県        Both   2010  01    Total      8266        
13  13             東京都        Both   2010  01    Total      22125       
14  14             神奈川県       Both   2010  01    Total      11362       
15  15             新潟県        Both   2010  01    Total      1111        
16  16             富山県        Both   2010  01    Total      688         
17  17             石川県        Both   2010  01    Total      815         
18  18             福井県        Both   2010  01    Total      477         
19  19             山梨県        Both   2010  01    Total      757         

   Outmigrants  
0   126018      
1   2561        
2   1119        
3   992         
4   2085        
5   753         
6   736         
7   1386        
8   2564        
9   1686        
10  1509        
11  8657        
12  7771        
13  21182       
14  11664       
15  1293        
16  674         
17  881         
18  520         
19  814

In [ ]:
def dataframe_to_postgres(df, tablename):
    cur = conn.cursor()
    textstream = io.StringIO()
    upload_df = df.copy()
    #upload_df['excel_description'].replace(['\n', '\t'], '', regex=True, inplace=True)

    upload_df.to_csv(textstream, sep='\t', header=False, index=False, quoting=csv.QUOTE_NONE)
    textstream.seek(0) 
    cur.copy_from(textstream, tablename, null="") # null values become ''
    conn.commit()

In [ ]:
#dataframe_to_postgres(in_migrants_age_df, 'jpstat_inmigrants')
dataframe_to_postgres(out_migrants_age_df, 'jpstat_outmigrants')

In [ ]:
#write using pyarrow with flavor=spark so that the files are compatible (datatypes)


In [ ]:
list_a = ['a', 'test', 'china', '2.xls']
list_a[-1]